## N-Body problems

Many physical problems require the evaluation of all pairwise interactions of a large number of particles, so-called N-body problems. These problems arise in molecular dynamics, astrodynamics and electromagnetics among others. 

Their pairwise interactions can be expressed as:

\begin{equation}
f_i = \sum_{j=1}^n{P \left(\boldsymbol{x}_i, \boldsymbol{x}_j \right)w_j} \ \ \ \text{for } i=1,2,...,n 
\end{equation}

*  where subscripts $i$,  $j$ respectively denote *target* and *source*
*  $f_i$ can be a *potential* (or *force*) at target point $i$
*  $w_j$ is the *source weight* 
*  $\boldsymbol{x}_i, \boldsymbol{x}_j$ are the *spatial positions* of particles 
*  $P \left(\boldsymbol{x}_i, \boldsymbol{x}_j \right)$ is the *interaction kernel*. 

In order to evalute the potential $f_i$ at a target point $i$, we have to loop over each source particle $j$. Since there are $n$ target points $i$, this 'brute-force' approach costs $\mathcal{O} \left(n^2 \right)$ operations. 

One possible approach in this kind of problem is to define a few classes, say `Point` and `Particle` and then loop over the objects and perform the necessary point-to-point calculations.  

In [27]:
import numpy

In [28]:
class Point():
    """    
    Arguments:
        domain: the domain of random generated coordinates x,y,z, 
                default=1.0
    
    Attributes:
        x, y, z: coordinates of the point
    """
    def __init__(self, domain=1.0):
        self.x = domain * numpy.random.random()
        self.y = domain * numpy.random.random()
        self.z = domain * numpy.random.random()
            
    def distance(self, other):
        return ((self.x - other.x)**2 + 
                (self.y - other.y)**2 + 
                (self.z - other.z)**2)**.5

In [29]:
class Particle(Point):
    """    
    Attributes:
        m: mass of the particle
        phi: the potential of the particle
    """
    
    def __init__(self, domain=1.0, m=1.0):
        Point.__init__(self, domain)
        self.m = m
        self.phi = 0.

Now we create a list of `n` random particles, define a function to calculate their interaction via direct summation and run!

In [30]:
n = 1000
particles = [Particle(m = 1 / n) for i in range(n)]

In [31]:
def direct_sum(particles):
    """
    Calculate the potential at each particle
    using direct summation method.

    Arguments:
        particles: the list of particles

    """
    for i, target in enumerate(particles):
        for source in (particles[:i] + particles[i+1:]):
            r = target.distance(source)
            target.phi += source.m / r

In [32]:
direct_sum(particles)

There was a noticeable lag there.  How long does this thing take for 1000 particles?

In [33]:
orig_time = %timeit -o direct_sum(particles)

981 ms ± 58.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [35]:
%lprun -f direct_sum direct_sum(particles)

## How do we use Numba on this problem?

Problem: Numba doesn't support jitting native Python classes.  There is a `jit_class` structure in Numba but it's still in early development.

But it's nice to have attributes for literate programming.

Solution: NumPy custom dtypes.

In [36]:
particle_dtype = numpy.dtype({'names':['x','y','z','m','phi'], 
                             'formats':[numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double]})

In [37]:
myarray = numpy.ones(3, dtype=particle_dtype)

In [38]:
myarray

array([(1., 1., 1., 1., 1.), (1., 1., 1., 1., 1.), (1., 1., 1., 1., 1.)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('m', '<f8'), ('phi', '<f8')])

You can access an individual "attribute" like this:

In [39]:
myarray[0]['x'] = 2.0

In [40]:
myarray

array([(2., 1., 1., 1., 1.), (1., 1., 1., 1., 1.), (1., 1., 1., 1., 1.)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('m', '<f8'), ('phi', '<f8')])

## [Exercise 1](./exercises/04.Direct.Summation.Exercises.ipynb#Exercise-1)

Write a function `create_n_random_particles` that takes the arguments `n` (number of particles), `m` (mass of every particle) and a domain within to generate a random number (as in the class above).
It should create an array with `n` elements and `dtype=particle_dtype` and then return that array.

For each particle, the mass should be initialized to the value of `m` and the potential `phi` initialized to zero.

In [41]:
from numba import njit

In [51]:
# %load snippets/nbody/create_n.py
@njit
def create_n_random_particles(n, m, domain):
    """Returns an array of n particles with mass 'm' and random position."""
    particles = numpy.zeros((n), dtype=particle_dtype)
    for particle in particles:
        particle['x'] = domain * numpy.random.random()
        particle['y'] = domain * numpy.random.random()
        particle['z'] = domain * numpy.random.random()
        particle['m'] = m
        particle['phi'] = 0.
    return particles

**Note**: You can use "attribute" access on dtypes but there's a caveat.  If you need to debug this function without the decorator, you have to change them back to array access form.  

In [52]:
parts = create_n_random_particles(1000, .001, 1)

In [53]:
parts[:3]

array([(0.65159553, 0.7557359 , 0.11180626, 0.001, 0.),
       (0.43769198, 0.67599745, 0.47245585, 0.001, 0.),
       (0.05187925, 0.19869663, 0.65753163, 0.001, 0.)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('m', '<f8'), ('phi', '<f8')])

We don't have a `distance` method anymore, so we need to write a function to take care of that.

## [Exercise 2](./exercises/04.Direct.Summation.Exercises.ipynb#Exercise-2)
Write a JITted function `distance` to calculate the distance between two particles of dtype `particle_dtype`

In [54]:
# %load snippets/nbody/distance.py
@njit
def distance(p1, p2):
    """Calculate distance between two particles."""
    return ((p1['x'] - p2['x'])**2 + 
            (p1['y'] - p2['y'])**2 +
            (p1['z'] - p2['z'])**2)**.5

In [55]:
distance(parts[0], parts[1])

0.4268267503083186

In [56]:
%%timeit
distance(parts[0], parts[1])

1.13 µs ± 25.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## [Exercise 3](./exercises/04.Direct.Summation.Exercises.ipynb#Exercise-3)
Modify the `direct_sum` function above to instead work a NumPy array of particles.  Loop over each element in the array and calculate its total potential.

In [76]:
# %load snippets/nbody/direct_sum.py
@njit
def direct_sum(particles):
    n = len(particles)
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            r = distance(particles[i], particles[j])
            particles[i]['phi'] += particles[j]['m'] / r

In [77]:
numba_time = %timeit -o direct_sum(parts)

4.72 ms ± 64.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
orig_time.best / numba_time.best

201.16911581665497